In [1]:
# Load pretrained model, and generate new transformed dataset by passing images through pretrained model 
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.datasets import cifar100
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
conv_base = VGG16(weights=None,include_top=False,input_shape=(32, 32, 3))
for layer in conv_base.layers:
    layer.trainable = True

In [3]:
# Instantiate model 
conv_base = VGG16(weights='imagenet',include_top=False,input_shape=(32, 32, 3))
for layer in conv_base.layers:
    layer.trainable = False

In [4]:
top_model = Sequential()
top_model.add(Flatten(input_shape=conv_base.output_shape[1:]))

top_model.add(Dense(512, activation='relu'))
top_model.add(BatchNormalization())
top_model.add(Dense(512, activation='relu'))
top_model.add(BatchNormalization())
top_model.add(Dense(100, activation='softmax'))
for layer in top_model.layers:
    layer.trainable = True 

final_model = Model(inputs=conv_base.input, outputs=top_model(conv_base.output))
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
num_classes = 100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [6]:
batch_size = 32
epochs = 25

datagen = ImageDataGenerator(rescale=1./255)
generator_train = datagen.flow(x_train, y_train, batch_size=batch_size)
generator_test = datagen.flow(x_test, y_test, batch_size=batch_size)

In [7]:
history = final_model.fit_generator(generator=generator_train,
                                    steps_per_epoch=x_train.shape[0] // batch_size,
                                    epochs=epochs,
                                    validation_data=generator_test,
                                    validation_steps=x_test.shape[0] // batch_size
                                   )

Epoch 1/25
1562/1562 [==============================] - 20s 13ms/step - loss: 3.1700 - acc: 0.2438 - val_loss: 2.8806 - val_acc: 0.2895
Epoch 2/25
1562/1562 [==============================] - 19s 12ms/step - loss: 2.6995 - acc: 0.3199 - val_loss: 2.7246 - val_acc: 0.3213
Epoch 3/25
1562/1562 [==============================] - 19s 12ms/step - loss: 2.5229 - acc: 0.3533 - val_loss: 2.6329 - val_acc: 0.3400
Epoch 4/25
1562/1562 [==============================] - 18s 12ms/step - loss: 2.3945 - acc: 0.3793 - val_loss: 2.6360 - val_acc: 0.3484
Epoch 5/25
1562/1562 [==============================] - 18s 12ms/step - loss: 2.2888 - acc: 0.4019 - val_loss: 2.6051 - val_acc: 0.3490
Epoch 6/25
1562/1562 [==============================] - 19s 12ms/step - loss: 2.1890 - acc: 0.4238 - val_loss: 2.6031 - val_acc: 0.3564
Epoch 7/25
1562/1562 [==============================] - 19s 12ms/step - loss: 2.0880 - acc: 0.4435 - val_loss: 2.6331 - val_acc: 0.3534
Epoch 8/25
1562/1562 [==========================